In [20]:
import pathlib
import os, sys
import numpy as np
from loguru import logger
from collections import defaultdict
from openset_imagenet.util import ccr_at_fpr
import openset_imagenet
from matplotlib import pyplot

In [21]:
def load_scores(args):
    # we sort them as follows: protocol, loss, algorithm
    scores = defaultdict(lambda: defaultdict(dict))
    ground_truths = {}
    for net in args["networks"]:
                output_directory = pathlib.Path(args["output_directory"])
                score_file = f"{net}.npz"
                if os.path.exists(score_file):
                    # remember files
                    results = np.load(score_file)
                    scores[net] = results["scores"] # only change in maxlogits case

                    if len(ground_truths) == 0:
                        ground_truths = results["gt"].astype(int)
                    else:
                        assert np.all(results["gt"] == ground_truths)

                    logger.info(f"Loaded score file {score_file} for net {net}")
                else:
                    logger.warning(f"Did not find score file {score_file} for net {net}")

    return scores, ground_truths

In [31]:
arguments = { 
  "output_directory": "experiments/ex_1",
  "networks": ["net_1", "net_2", "net_3", "net_4", "net_5", "net_6"]
  }
THRESHOLDS = {
              1e-3: "$10^{-3}$",
              1e-2: "$10^{-2}$",
              1e-1: "$10^{-1}$",
              1: "$1$",
}

scores, ground_truths = load_scores(arguments)
# we get ccr@fpr for each network
for net in arguments["networks"]:
    ccrs = ccr_at_fpr(ground_truths, scores[net], THRESHOLDS)
    print(f"Network {net}: {ccrs} for thresholds {THRESHOLDS.values()}")

2024-05-08 13:34:18.946 | INFO     | __main__:load_scores:18 - Loaded score file net_1.npz for net net_1
2024-05-08 13:34:18.951 | INFO     | __main__:load_scores:18 - Loaded score file net_2.npz for net net_2
2024-05-08 13:34:18.955 | INFO     | __main__:load_scores:18 - Loaded score file net_3.npz for net net_3
2024-05-08 13:34:18.958 | INFO     | __main__:load_scores:18 - Loaded score file net_4.npz for net net_4
2024-05-08 13:34:18.960 | INFO     | __main__:load_scores:18 - Loaded score file net_5.npz for net net_5
2024-05-08 13:34:18.963 | INFO     | __main__:load_scores:18 - Loaded score file net_6.npz for net net_6


Network net_1: [0.0009, 0.0354, 0.5452, 0.9853] for thresholds dict_values(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$', '$1$'])
Network net_2: [None, None, 0.5214, 0.9865] for thresholds dict_values(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$', '$1$'])
Network net_3: [0.0005, 0.0384, 0.5198, 0.9872] for thresholds dict_values(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$', '$1$'])
Network net_4: [None, 0.0276, 0.4914, 0.9866] for thresholds dict_values(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$', '$1$'])
Network net_5: [0.003, 0.04, 0.5076, 0.9875] for thresholds dict_values(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$', '$1$'])
Network net_6: [0.0001, 0.0206, 0.5401, 0.9871] for thresholds dict_values(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$', '$1$'])
